In [30]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time
from datetime import date, timedelta
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
import cloudscraper
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',200)
pd.set_option('display.max_colwidth',300)
import warnings
warnings.filterwarnings("ignore")

# --------------------------------------------------

In [ ]:
nejm = 'https://www.nejm.org/coronavirus'
corona = requests.get(nejm)
pag_princ = bs(corona.content, "html.parser")

title = []
link = []
des = []
autor = []
data = []
for contain in pag_princ.find_all("ul", {"o-curated-area"}):
    try:
        title_row = contain.b.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
        link_row = contain.find('a', {'class': 'm-article__link'})['href']
        des_row = contain.p.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
        autor_row = contain.find('em', {'class': 'm-article__author f-author'}).get_text()
        data_row = contain.find('em', {'class': 'm-article__date f-tag'}).get_text()
        title.append(title_row)
        link.append(link_row)
        des.append(des_row)
        autor.append(autor_row)
        data.append(data_row)
        #print("title: {}\nlink: {}\ndescription: {}\n".format(title,link,des))
    except Exception as e:
        print('Error, skipping...', e)
for contain in pag_princ.find_all("li", {"m-result m-result--img separatorLines"}):
    try:
        title_row = contain.strong.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
        link_row = contain.find('a', {'class': 'm-result__link'})['href']
        des_row = contain.find('span', {'class': 'm-result__blurb f-blurb'}).find('p').get_text()
        autor_row = contain.find('em', {'class': 'm-result__author f-author'}).get_text()
        data_row = contain.find('em', {'class': 'm-result__date f-tag'}).get_text()
        title.append(title_row)
        link.append(link_row)
        des.append(des_row)
        autor.append(autor_row)
        data.append(data_row)
        #print("title: {}\nlink: {}\ndescription: {}\n".format(title,link,des))
    except Exception as e:
        print('Error, skipping...', e)        
         
df_corona_news = pd.DataFrame({'Data':data,'Titulo':title,'Descricao':des,'Autor':autor,'Link':link })
df_corona_news = df_corona_news.astype(str)
df_corona_news['Descricao'] = df_corona_news['Descricao'].str.replace('\n ', '')
df_corona_news['Titulo'] = df_corona_news['Titulo'].str.replace('\n ', '')
df_corona_news['Descricao'] = df_corona_news['Descricao'].str.strip()
df_corona_news['Titulo'] = df_corona_news['Titulo'].str.strip()
df_corona_news['Doi'] = df_corona_news['Link'].str.replace('/doi/full/', '')
df_corona_news['Doi'] = df_corona_news['Doi'].str.replace('?', '')
df_corona_news['Doi'] = df_corona_news['Doi'].str.replace('query=featured_coronavirus', '')
df_corona_news['Doi'] = df_corona_news['Doi'].str.replace('query=featured_home', '')
df_corona_news = df_corona_news[['Data','Titulo','Descricao','Autor','Doi','Link']]
df_corona_news['Link'] = 'https://www.nejm.org' + df_corona_news['Link'].astype(str)
df_corona_news['Fonte'] = 'NEJM'
news_1 = df_corona_news
news_1

In [ ]:
# df_corona_news.to_csv('The_New_England_Journal_of_Medicine.csv', sep = '|', encoding='utf-8')

# -------------------------------------------------------

In [ ]:
ua = UserAgent()
header = {
    "User-Agent": ua.random
}
r = requests.get('https://www.jamanetwork.com/journals/jama/pages/coronavirus-alert', headers=header)

pag_princ = bs(r.content, "html.parser")

title = []
link = []
# des = []
autor = []
# data = []
for contain in pag_princ.find_all("div", {"alert-info"}):
    try:
        title_row = contain.a.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
        link_row = contain.a["href"]
#         des_row = contain.p.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
        autor_row = contain.find('span')
#         data_row = contain.find('em', {'class': 'm-article__date f-tag'}).get_text()
        title.append(title_row)
        link.append(link_row)
#         des.append(des_row)
        autor.append(autor_row)
#         data.append(data_row)
        #print("title: {}\nlink: {}\ndescription: {}\n".format(title,link,des))
    except Exception as e:
        print('Error, skipping...', e)  
         
df_corona_news = pd.DataFrame({'Titulo':title,'Autor':autor,'Link':link })
df_corona_news['Autor'] = df_corona_news['Autor'].astype(str)
df_corona_news['Autor'] = df_corona_news['Autor'].str.replace('<span>', '')
df_corona_news['Autor'] = df_corona_news['Autor'].str.replace('</span>', '')
df_corona_news = df_corona_news[df_corona_news['Link'].str.contains('/journals')]
df_corona_news['Link'] = 'https://jamanetwork.com' + df_corona_news['Link'].astype(str)
df_corona_news['Fonte'] = 'JAMA'
news_2 = df_corona_news
news_2

In [ ]:
# df_corona_news.to_csv('Jama_Network.csv', sep = '|', encoding='utf-8')

# -----------------------------------------------------

In [ ]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

s = requests.Session()
s.headers.update({'x-test': 'true'})


title = []
link = []
# des = []
autor = []
data = []

i=-1
while True:
    i = i+1
    url = "https://www.thelancet.com/coronavirus/archive?startPage=" + str(i)
    corona = requests_retry_session(session=s).get(url)
    if corona.status_code != 200:
        break
    corona = requests_retry_session(session=s).get(url)
    pag_princ = bs(corona.content, "html.parser")
#     print('Lendo página...', i)
    if pag_princ.find_all("div", {"articleCitation"}) == []:
        break
    for contain in pag_princ.find_all("div", {"articleCitation"}):
        try:
            title_row = contain.a.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
            link_row = contain.a["href"]
    #         des_row = contain.p.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
            autor_row = contain.find('div', {'class': 'authors'}).get_text()
            data_row = contain.find('div', {'class': 'published-online'}).get_text()
            title.append(title_row)
            link.append(link_row)
    #         des.append(des_row)
            autor.append(autor_row)
            data.append(data_row)
#             print("data: {}\ntitle: {}\nautor: {}\nlink: {}\n".format(data,title,autor,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Data':data,'Titulo':title,'Autor':autor,'Link':link })
df_corona_news['Autor'] = df_corona_news['Autor'].astype(str)
df_corona_news['Autor'] = df_corona_news['Autor'].str.replace('<span>', '')
df_corona_news['Autor'] = df_corona_news['Autor'].str.replace('</span>', '')
df_corona_news['Data'] = df_corona_news['Data'].astype(str)
df_corona_news['Data'] = df_corona_news['Data'].str.replace('Published:', '')
df_corona_news['Link'] = 'https://www.thelancet.com' + df_corona_news['Link'].astype(str)
df_corona_news['Fonte'] = 'LANCET'
news_3 = df_corona_news
news_3

In [ ]:
# df_corona_news.to_csv('The_Lancet.csv', sep = '|', encoding='utf-8')

# -------------------------------------------------

In [ ]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

title = []
link = []
# des = []
autor = []
# data = []

s = requests.Session()
s.headers.update({'x-test': 'true'})
url = "https://www.cell.com/2019-nCOV"
corona = requests_retry_session(session=s).get(url)
pag_princ = bs(corona.content, "html.parser")

for contain in pag_princ.find_all("div", {"article-details"}):
        try:
            title_row = contain.a.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
            link_row = contain.a["href"]
    #         des_row = contain.p.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
            autor_row = contain.find('div', {'class': 'authors'}).get_text()
#             data_row = contain.find('div', {'class': 'published-online'}).get_text()
            title.append(title_row)
            link.append(link_row)
    #         des.append(des_row)
            autor.append(autor_row)
#             data.append(data_row)
#             print("title: {}\nautor: {}\nlink: {}\n".format(title,autor,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Titulo':title,'Autor':autor,'Link':link })
df_corona_news['Autor'] = df_corona_news['Autor'].astype(str)
df_corona_news['Autor'] = df_corona_news['Autor'].str.replace('<span>', '')
df_corona_news['Autor'] = df_corona_news['Autor'].str.replace('</span>', '')
df_corona_news.loc[~df_corona_news['Link'].str.contains('//els-jbs-prod-cdn|https'), 'Link'] = 'https://www.cell.com' + df_corona_news['Link'].astype(str)
df_corona_news['Link'] = df_corona_news['Link'].str.replace(':443', '')
df_corona_news.loc[~df_corona_news['Link'].str.contains('https'), 'Link'] = 'https:' + df_corona_news['Link'].astype(str)
df_corona_news['Fonte'] = 'CELL'
news_4 = df_corona_news
news_4

In [ ]:
# df_corona_news.to_csv('Cell.csv', sep = '|', encoding='utf-8')

# ----------------------------------------

In [ ]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

title = []
link = []
# des = []
# autor = []
data = []

s = requests.Session()
s.headers.update({'x-test': 'true'})
url = "https://www.bmj.com/archive/sevendays"
corona = requests_retry_session(session=s).get(url)
pag_princ = bs(corona.content, "html.parser")

for contain in pag_princ.find_all("div", {"highwire-cite highwire-cite-highwire-article highwire-citation-bmj-toc-list clearfix"}):
        try:
            title_row = contain.find('span', {'class': 'highwire-cite-title'}).get_text()
            link_row = contain.find('a', {'class': 'highwire-cite-linked-title'})['href']
    #         des_row = contain.p.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
#             autor_row = contain.find('div', {'class': 'authors'}).get_text()
            data_row = contain.find('span', {'class': 'highwire-cite-metadata-date highwire-cite-metadata'}).find_next_sibling("span").find_next_sibling("span").find_next_sibling("span").get_text()
            title.append(title_row)
            link.append(link_row)
    #         des.append(des_row)
#             autor.append(autor_row)
            data.append(data_row)
#             print("data: {}\ntitle: {}\nlink: {}\n".format(data,title,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Data':data,'Titulo':title,'Link':link })
df_corona_news['Data'] = df_corona_news['Data'].astype(str)
df_corona_news['Data'] = df_corona_news['Data'].str.replace('(', '')
df_corona_news['Data'] = df_corona_news['Data'].str.replace(')', '')
df_corona_news['Data'] = df_corona_news['Data'].str.replace('Published', '')
df_corona_news['Link'] = 'https://www.bmj.com' + df_corona_news['Link'].astype(str)
df_corona_news = df_corona_news[df_corona_news['Titulo'].str.contains('covid-19|Covid-19|coronavirus|Coronavirus')]
df_corona_news['Fonte'] = 'BMJ'
news_5 = df_corona_news
news_5

In [ ]:
# df_corona_news.to_csv('bmj.csv', sep = '|', encoding='utf-8')

# ----------------------------------

In [ ]:
def requests_retry_session(
    retries=10,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session
ua = UserAgent()
s = requests.Session()
s.headers.update({"User-Agent": ua.random})


title = []
link = []
des = []
autor = []
data = []

i=0
while True:
    i = i+1
    url = "https://academic.oup.com/journals/search-results?q=coronavirus&sort=Date+–+Newest+First&allJournals=1&fl_SiteID=5567&page=" + str(i)
    corona = requests_retry_session(session=s).get(url)
    if corona.status_code != 200:
        break
    corona = requests_retry_session(session=s).get(url)
    pag_princ = bs(corona.content, "html.parser")
    print('Lendo página...', i)
    if pag_princ.find_all("div", {"sr-list al-article-box al-normal clearfix"}) == []:
        break
    for contain in pag_princ.find_all("div", {"sr-list al-article-box al-normal clearfix"}):
        try:
            title_row = contain.find('a', {'class': 'article-link'}).get_text()
            link_row = contain.a["href"]
            des_row = contain.find('div', {'class': 'snippet'}).get_text()
            autor_row = contain.find('div', {'class': 'sri-authors al-authors-list'}).get_text()
            data_row = contain.find('div', {'class': 'sri-date al-pub-date'}).get_text()
            title.append(title_row)
            link.append(link_row)
            des.append(des_row)
            autor.append(autor_row)
            data.append(data_row)
#             print("data: {}\ntitle: {}\ndes: {}\nautor: {}\nlink: {}\n".format(data,title,des,autor,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Data':data,'Titulo':title,'Descricao':des,'Autor':autor,'Link':link })
df_corona_news['Descricao'] = df_corona_news['Descricao'].astype(str)
df_corona_news['Descricao'] = df_corona_news['Descricao'].str.replace('\r', '')
df_corona_news['Descricao'] = df_corona_news['Descricao'].str.replace('\n', '')
df_corona_news['Data'] = df_corona_news['Data'].astype(str)
df_corona_news['Data'] = df_corona_news['Data'].str.replace('Published:', '')
df_corona_news['Link'] = 'https://academic.oup.com' + df_corona_news['Link'].astype(str)
df_corona_news['Fonte'] = 'OXFORD'
news_6 = df_corona_news
news_6

In [ ]:
# df_corona_news.to_csv('oxford.csv', sep = '|', encoding='utf-8')

# -----------------------------------------------

In [ ]:
def requests_retry_session(
    retries=10,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

title = []
link = []
# des = []
# autor = []
#data = []

s = cloudscraper.CloudScraper()
ua = UserAgent()
s.headers.update({"User-Agent": ua.random})
url = "https://www.cdc.gov/coronavirus/2019-ncov/communication/publications.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fcoronavirus%2F2019-ncov%2Fpublications.html"
corona = requests_retry_session(session=s).get(url)
pag_princ = bs(corona.content, "html.parser")

ul = pag_princ.find("p", text="COVID-19 MMWRs").find_next_sibling("ul")
for contain in ul.find_all('li'):
        try:
            title_row = contain.get_text()
            link_row = contain.find('a', {'class': 'tp-link-policy'})['href']
    #         des_row = contain.p.text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
#             autor_row = contain.find('div', {'class': 'authors'}).get_text()
#             data_row = contain.find('span', {'class': 'highwire-cite-metadata-date highwire-cite-metadata'}).find_next_sibling("span").find_next_sibling("span").find_next_sibling("span").get_text()
            title.append(title_row)
            link.append(link_row)
    #         des.append(des_row)
#             autor.append(autor_row)
#             data.append(data_row)
#             print("title: {}\nlink: {}\n".format(title,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Titulo_full':title,'Link':link})
df_corona_news = df_corona_news.join(df_corona_news['Titulo_full'].str.split('.', expand=True).rename(columns={0:'Autor', 1:'Titulo'}))
df_corona_news = df_corona_news[['Titulo','Autor','Link']]
df_corona_news['Fonte'] = 'CDC'
news_7 = df_corona_news
news_7

In [ ]:
# df_corona_news.to_csv('CDC.csv', sep = '|', encoding='utf-8')

# ----------------------------------------------------

In [ ]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

title = []
link = []
des = []
autor = []
data = []

s = requests.Session()
s.headers.update({'x-test': 'true'})
url = "https://www.nature.com/collections/hajgidghjb"
corona = requests_retry_session(session=s).get(url)
pag_princ = bs(corona.content, "html.parser")

for contain in pag_princ.find_all("article", {"cleared c-article-item flex-box flex-rr-nowrap mq640-kill-flex"}):
        try:
            title_row = contain.a.get_text(strip=True)
            link_row = contain.a["href"]
            des_row = contain.find('div', {'class': 'c-article-item__description tighten-line-height'}).get_text(strip=True)
            autor_row = contain.find('ul', {'class': 'c-article-item__authors'}).get_text(strip=True)
            data_row = contain.find('time', {'class': 'c-article-item__publication-date'}).get_text(strip=True)
            title.append(title_row)
            link.append(link_row)
            des.append(des_row)
            autor.append(autor_row)
            data.append(data_row)
#             print("data: {}\ntitle: {}\ndes: {}\nautor: {}\nlink: {}\n".format(data,title,des,autor,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Data':data,'Titulo':title,'Descricao':des,'Autor':autor,'Link':link })
df_corona_news['Link'] = 'https://www.nature.com' + df_corona_news['Link'].astype(str)
df_corona_news['Fonte'] = 'NATURE'
news_8 = df_corona_news
news_8

In [ ]:
# df_corona_news.to_csv('Nature.csv', sep = '|', encoding='utf-8')

# ----------------------------------------------

In [ ]:
def requests_retry_session(
    retries=10,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

title = []
link = []
doi = []
autor = []
# data = []

s = cloudscraper.CloudScraper()
s.headers.update({'x-test': 'true'})
url = "https://novel-coronavirus.onlinelibrary.wiley.com/"
corona = requests_retry_session(session=s).get(url)
pag_princ = bs(corona.content, "html.parser")

for contain in pag_princ.find_all("div", {"issue-item"}):
        try:
            title_row = contain.find('a', {'class': 'issue-item__title visitable'}).get_text(strip=True)
            link_row = contain.a["href"]
            doi_row = contain.find('li', {'class': 'coverDoi'}).get_text(strip=True)
            autor_row = contain.find('ul', {'class': 'rlist--inline loa comma'}).get_text(strip=True)
# #             data_row = contain.find('time', {'class': 'c-article-item__publication-date'}).get_text(strip=True)
            title.append(title_row)
            link.append(link_row)
            doi.append(doi_row)
            autor.append(autor_row)
#             data.append(data_row)
#             print("title: {}\ndoi: {}\nautor: {}\nlink: {}\n".format(title,doi,autor,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Titulo':title,'Autor':autor,'Doi':doi,'Link':link })
df_corona_news['Doi'] = df_corona_news['Doi'].astype(str)
df_corona_news['Doi'] = df_corona_news['Doi'].str.replace('DOI:&nbsp', '')
df_corona_news['Fonte'] = 'WILLEY'
news_9 = df_corona_news
news_9

In [ ]:
# df_corona_news.to_csv('Novel_Coronavirus_Wiley_Online_Library.csv', sep = '|', encoding='utf-8')

# ------------------------------------------------

In [ ]:
def requests_retry_session(
    retries=10,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session
ua = UserAgent()
s = requests.Session()
s.headers.update({"User-Agent": ua.random})


title = []
link = []
# des = []
autor = []
data = []

i=0
while True:
    i = i+1
    url = "https://www.cambridge.org/core/browse-subjects/medicine/coronavirus-free-access-collection?pageNum=" + str(i)
    corona = requests_retry_session(session=s).get(url)
    if corona.status_code != 200:
        break
    corona = requests_retry_session(session=s).get(url)
    pag_princ = bs(corona.content, "html.parser")
    print('Lendo página...', i)
    if pag_princ.find_all("ul", {"details"}) == []:
        break
    for contain in pag_princ.find_all("ul", {"details"}):
        try:
            title_row = contain.find('a', {'class': 'part-link'}).get_text(strip=True)
            link_row = contain.a["href"]
#             des_row = contain.find('div', {'class': 'snippet'}).get_text()
            autor_row = contain.find('a', {'class': 'more-by-this-author'}).get_text(strip=True)
            data_row = contain.find('span', {'class': 'date'}).get_text(strip=True)
            title.append(title_row)
            link.append(link_row)
#             des.append(des_row)
            autor.append(autor_row)
            data.append(data_row)
#             print("data: {}\ntitle: {}\nautor: {}\nlink: {}\n".format(data,title,autor,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Data':data,'Titulo':title,'Autor':autor,'Link':link })
df_corona_news['Link'] = 'https://www.cambridge.org' + df_corona_news['Link'].astype(str)
df_corona_news['Fonte'] = 'CAMBRIDGE'
news_10 = df_corona_news
news_10

In [ ]:
# df_corona_news.to_csv('Cambridge.csv', sep = '|', encoding='utf-8')

# ---------------------------------

In [ ]:
def requests_retry_session(
    retries=10,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session
ua = UserAgent()
s = requests.Session()
s.headers.update({"User-Agent": ua.random})


title = []
link = []
doi = []
autor = []
data = []

i=0
while True:
    i = i+1
    url = "http://connect.medrxiv.org/relate/content/181?page=" + str(i)
    corona = requests_retry_session(session=s).get(url)
    if corona.status_code != 200:
        break
    corona = requests_retry_session(session=s).get(url)
    pag_princ = bs(corona.content, "html.parser")
    print('Lendo página...', i)
    if i==213:
        break
    for contain in pag_princ.find_all("div", {"highwire-article-citation highwire-citation-type-highwire-article"}):
        try:
            title_row = contain.find('a', {'class': 'highwire-cite-linked-title'}).get_text(strip=True)
            link_row = contain.a["href"]
#             des_row = contain.find('div', {'class': 'snippet'}).get_text()
#             autor_row = contain.find('span', {'class': 'highwire-citation-authors'})
#             data_row = contain.find('span', {'class': 'highwire-cite-metadata-journal'}).get_text(strip=True)
            title.append(title_row)
            link.append(link_row)
#             des.append(des_row)
#             autor.append(autor_row)
#             data.append(data_row)
#             print("data: {}\ntitle: {}\nautor: {}\nlink: {}\n".format(data,title,autor,link))
        except Exception as e:
            print('Error, skipping...', e)
            
    for contain in pag_princ.find_all("div", {"highwire-cite-authors"}):
        try:
            autor_row = contain.find('span', {'class': 'highwire-citation-authors'}).get_text(strip=True)
            autor.append(autor_row)
#             print("data: {}\ntitle: {}\nautor: {}\nlink: {}\n".format(data,title,autor,link))
        except Exception as e:
            print('Error, skipping...', e)
    for contain in pag_princ.find_all("div", {"highwire-cite-metadata"}):
        try:
            doi_row = contain.find('span', {'class': 'highwire-cite-metadata-journal'}).find('a').get_text(strip=True)
            data_row = contain.find('span', {'class': 'highwire-cite-metadata-journal'}).get_text(strip=True)
            doi.append(doi_row)
            data.append(data_row)
#             print("data: {}\ntitle: {}\ndoi: {}nautor: {}\nlink: {}\n".format(data,title,doi,autor,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Data':data,'Titulo':title,'Doi':doi,'Autor':autor,'Link':link })
df_corona_news['Data'] = df_corona_news['Data'].str[-10:]
df_corona_news['Fonte'] = 'medRxiv'
news_11 = df_corona_news
news_11

In [ ]:
# df_corona_news.to_csv('medrxiv.csv', sep = '|', encoding='utf-8')

# -----------------------------------------

In [ ]:
def requests_retry_session(
    retries=10,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


title = []
link = []
# des = []
# autor = []
data = []

i=-1
while True:
    i = i+1
    ua = UserAgent()
    s = requests.Session()
    s.headers.update({'User-Agent':str(ua.random)})
    url = "https://www.ecdc.europa.eu/en/coronavirus?type%5B0%5D=1244&type%5B1%5D=1307&type%5B2%5D=1382&sort_by=field_ct_publication_date_value&items_per_page=4&pager_type=infinite_scroll&type_op=or&tid_op=or&tid%5B0%5D%5Btarget_id%5D=2943&sort_order=DESC&bid=MMgA0tBXpTBOCUYtiHvtZZMke0deSswQEdBwXsQNaQM&nid=33355&page=" + str(i)
    corona = requests_retry_session(session=s).get(url)
    if corona.status_code != 200:
        break
    corona = requests_retry_session(session=s).get(url)
    pag_princ = bs(corona.content, "html.parser")
    pag_princ = pag_princ.find("div", {"col-xs-12 col-sm-10 col-sm-offset-1"})
    print('Lendo página...', i)
    if pag_princ.find_all("article", {"ct--view-15 node node--type-ct-publication node--view-mode-vm-view15 ct"}) == []:
        break
    for contain in pag_princ.find_all("article", {"ct--view-15 node node--type-ct-publication node--view-mode-vm-view15 ct"}):
        try:
            title_row = contain.find('h3', {'class': 'ct__title'}).get_text(strip=True)
            link_row = contain.find('a', {'class': 'ct__link'})["href"]
#             des_row = contain.find('div', {'class': 'c-article-item__description tighten-line-height'}).get_text(strip=True)
#             autor_row = contain.find('ul', {'class': 'c-article-item__authors'}).get_text(strip=True)
            data_row = contain.find('time', {'class': 'ct__meta__value'}).get_text(strip=True)
            title.append(title_row)
            link.append(link_row)
#             des.append(des_row)
#             autor.append(autor_row)
            data.append(data_row)
#             print("data: {}\ntitle: {}\nlink: {}".format(data,title,link))
        except Exception as e:
            print('Error, skipping...', e)

df_corona_news = pd.DataFrame({'Data':data,'Titulo':title,'Link':link })
df_corona_news = df_corona_news.drop_duplicates()
df_corona_news['Link'] = 'https://www.ecdc.europa.eu' + df_corona_news['Link'].astype(str)
df_corona_news['Fonte'] = 'ECDC'
news_12 = df_corona_news
news_12

In [ ]:
# df_corona_news.to_csv('ECDC.csv', sep = '|', encoding='utf-8')

In [ ]:
full_news = pd.concat([news_1,news_2,news_3,news_4,news_5,news_6,news_7,news_8,news_9,news_10,news_11,news_12]).reset_index(drop = True)
full_news = full_news[['Data','Titulo','Descricao','Autor','Doi','Link','Fonte','Responsavel']]
full_news = full_news.applymap(str)
full_news = full_news.applymap(str.strip)
full_news['count'] = pd.isnull(full_news).sum(1)
full_news = full_news.sort_values(['count']).drop_duplicates(subset=['Titulo'],keep='first').drop('count',1)
full_news = full_news.sort_values("Fonte")
full_news

In [ ]:
today = pd.Timestamp('today')
full_news.to_csv('covida_news_{:%m%d%Y}'.format(today)+'.csv', sep = ';', encoding='utf_8')

In [ ]:
today = pd.Timestamp('today')
full_news.to_excel(r'covida_news_{:%m%d%Y}'.format(today)+'.xlsx',sheet_name= 'covida_news_{:%m%d%Y}'.format(today))

In [32]:
yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
yesterday

'2020-05-27'